In [1]:
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import time
import glob
from lxml.html import parse
from sphfile import SPHFile
import pydub
import audiosegment
import pandas as pd
from collections import Counter
from bs4 import BeautifulSoup
import sys
root = '../'

## Split

In [2]:
segments_df = pd.read_csv(root+'files/segments.csv')
segments_df = segments_df.sample(frac=1, random_state = 12345)

speakers = Counter(segments_df['speaker'])
common_speakers = speakers.most_common(20)
print(common_speakers)
#common_speakers = [common_speaker[0] for common_speaker in common_speakers]


[('joie_chen', 5360), ('peter_jennings', 5000), ('lou_waters', 3838), ('linden_soles', 3354), ('leon_harris', 3099), ('david_brancaccio', 2872), ('ted_koppel', 2783), ('natalie_allen', 2750), ('karen_maginnis', 2742), ('brian_lamb', 2590), ('noah_adams', 2018), ('jeanne_meserve', 2009), ('bill_clinton', 1899), ('lynn_vaughan', 1887), ('kathleen_kennedy', 1778), ('thalia_assuras', 1587), ('mark_mullen', 1483), ('byron_miranda', 1477), ('lisa_mullins', 1443), ('csp_waj_susan', 1371)]


In [ ]:
# make train.csv
train_df = pd.DataFrame()

## Train

## Maybe try increasing dataset size by doing each combination of 290*290

In [4]:
# segments_by_speakers = {speaker:segments_df[segments_df['speaker']==speaker][:1271] for speaker in common_speakers}
# overlay = []
# for first_speaker in common_speakers:
#     second_speakers = [speaker for speaker in common_speakers if speaker!=first_speaker ]
#     first_segments = segments_by_speakers[first_speaker]
#     for second_speaker in second_speakers:
#         second_segments = segments_by_speakers[second_speaker]
#         first_idx = np.arange(200)
#         second_idx = np.arange(200)
#         np.random.shuffle(first_idx)
#         np.random.shuffle(second_idx)
#         for i in range(200):
#             first_segment = first_segments.iloc[first_idx[i]]
#             second_segment = second_segments.iloc[second_idx[i]]
#             row = {'first_speaker': first_speaker, 'second_speaker': second_speaker,
#                   'first_file': first_segment['segfile'], 'second_file': second_segment['segfile']}
#             overlay.append(row)
# overlay = pd.DataFrame(overlay)


In [5]:
overlay_shuffled = overlay.sample(frac=1, random_state = 12345)
overlay_shuffled.to_csv('overlay-train.csv')

## Val

In [7]:
segments_by_speakers = {speaker:segments_df[segments_df['speaker']==speaker][200:246] for speaker in common_speakers}
overlay = []
for first_speaker in common_speakers:
    second_speakers = [speaker for speaker in common_speakers if speaker!=first_speaker ]
    first_segments = segments_by_speakers[first_speaker]
    for second_speaker in second_speakers:
        second_segments = segments_by_speakers[second_speaker]
        first_idx = np.arange(46)
        second_idx = np.arange(46)
        np.random.shuffle(first_idx)
        np.random.shuffle(second_idx)
        for i in range(46):
            first_segment = first_segments.iloc[first_idx[i]]
            second_segment = second_segments.iloc[second_idx[i]]
            row = {'first_speaker': first_speaker, 'second_speaker': second_speaker,
                  'first_file': first_segment['segfile'], 'second_file': second_segment['segfile']}
            overlay.append(row)
overlay = pd.DataFrame(overlay)

In [8]:
overlay_shuffled = overlay.sample(frac=1, random_state = 12345)
overlay_shuffled.to_csv('overlay-val.csv')

## Test

In [9]:
segments_by_speakers = {speaker:segments_df[segments_df['speaker']==speaker][246:292] for speaker in common_speakers}
overlay = []
for first_speaker in common_speakers:
    second_speakers = [speaker for speaker in common_speakers if speaker!=first_speaker ]
    first_segments = segments_by_speakers[first_speaker]
    for second_speaker in second_speakers:
        second_segments = segments_by_speakers[second_speaker]
        first_idx = np.arange(46)
        second_idx = np.arange(46)
        np.random.shuffle(first_idx)
        np.random.shuffle(second_idx)
        for i in range(46):
            first_segment = first_segments.iloc[first_idx[i]]
            second_segment = second_segments.iloc[second_idx[i]]
            row = {'first_speaker': first_speaker, 'second_speaker': second_speaker,
                  'first_file': first_segment['segfile'], 'second_file': second_segment['segfile']}
            overlay.append(row)
overlay = pd.DataFrame(overlay)

In [10]:
overlay_shuffled = overlay.sample(frac=1, random_state = 12345)
overlay_shuffled.to_csv('overlay-test.csv')